### Imports

#### Import Packages

In [1]:
import polars as pl
import altair as alt
import numpy as np
from great_tables import GT
import gc

# alt.JupyterChart.enable_offline()
# alt.renderers.enable("jupyter", offline=True)

#### Import Panel Data

In [2]:
grocery_lf = pl.scan_csv(source="data/panel-datasets/edible_grocery.csv",
                         has_header=True,
                         separator=",",
                         schema={'panel_id': pl.Int32,
                                 'trans_id': pl.Int32,
                                 'week': pl.Int16,
                                 'sku_id': pl.Categorical,
                                 'units': pl.Int16,
                                 'price': pl.Float32,
                                 'brand': pl.Categorical})
grocery_lf.head().collect()

panel_id,trans_id,week,sku_id,units,price,brand
i32,i32,i16,cat,i16,f32,cat
3102011,1569,6,"""19""",1,2.79,"""Alpha"""
3102012,4301,15,"""15""",1,3.19,"""Alpha"""
3102012,4301,15,"""15""",1,3.19,"""Alpha"""
3102012,4301,15,"""38""",1,3.49,"""Bravo"""
3102012,4301,15,"""44""",1,3.49,"""Bravo"""


In [3]:
sku_lf = pl.scan_csv(source="data/panel-datasets/sku_weight.csv",
                         has_header=True,
                         separator=",",
                         schema={'sku_id': pl.Categorical,
                                 'weight': pl.Int16})
sku_lf.head().collect()

sku_id,weight
cat,i16
"""1""",400
"""2""",400
"""3""",400
"""4""",250
"""5""",1000


In [4]:
kiwi_lf = pl.scan_csv(source="data/panel-datasets/kiwibubbles_trans.csv",
                      has_header=True,
                      separator=",",
                      schema={'ID': pl.Int16,
                              'Market': pl.Categorical,
                              'Week': pl.Int16,
                              'Day': pl.Int16,
                              'Units': pl.Int16})
kiwi_lf.head().collect()

ID,Market,Week,Day,Units
i16,cat,i16,i16,i16
10001,"""1""",19,3,1
10002,"""1""",12,5,1
10003,"""1""",37,7,1
10004,"""1""",30,6,1
10004,"""1""",47,3,1


#### Reusable Functions

In [5]:
def weekly_plot(dataframe, y, color=None, title="", y_axis_label="", pct=False, legend=False):
    
    # Configure the color encoding only if color is provided
    if color is not None:
        color_encoding = alt.Color(
            f'{color}:N',  # N = a discrete unordered category
            legend=alt.Legend(title=color) if legend else None  # Add legend conditionally
        )
    else:
        color_encoding = alt.Color()  # No color encoding    
    
    chart = alt.Chart(dataframe).mark_line(strokeWidth=1).encode(
        x = alt.X(
            'week',
            axis=alt.Axis(
                values=np.arange(0, 104 + 1, 13), # Explicitly specify quarter-end weeks
                labelExpr="datum.value", # Show only these labels
                title='Week'
            )
        ),
        y = alt.Y(
            f'{y}:Q', # Q = a continuous real-valued quantity
            title=y_axis_label,
            axis=alt.Axis(format="$,.0f") if not pct else alt.Axis(format=",.0%")
        ),
        color = color_encoding
    ).properties(
        width=650,
        height=250,
        title=title
    ).configure_view(
        stroke=None
    ).configure_axisY(
        # grid=False # turn off y-axis grid if required
    )

    return alt.JupyterChart(chart)

### Preliminaries

#### Weekly Sales Pattern

In [6]:
# Grocery Sales LazyFrame (Query Plan): Category Weekly 'spend' - units x price = spend
grocery_spend_category = (
    grocery_lf
    .select(
        ['week', 'units', 'price']
    )
    .with_columns(
        ((pl.col('units') * pl.col('price')).cast(pl.Float64)).alias('spend')
    )
    .group_by('week')
    .agg(
        pl.col("spend").sum().cast(pl.Float64).alias('Weekly Spend')
    )
    .sort('week')
)

In [7]:
# Grocery Sales LazyFrame (Query Plan): Weekly 'spend' by Brand - units x price = spend
grocery_spend_brand = (
    grocery_lf
    .select(
        ['week', 'units', 'price', 'brand']
    )
    .with_columns(
         ((pl.col('units') * pl.col('price')).cast(pl.Float64)).alias('spend')
    )
    .group_by('week', 'brand')
    .agg(
        pl.col("spend").sum().cast(pl.Float64).alias('Weekly Spend')
    )
    .sort('week')
)

In [8]:
# Weekly Sales Pivot Table - Polars DataFrame
# For visualizing and inspecting only
weekly_sales_pivot = grocery_spend_brand.collect().pivot(
    on="brand",
    index="week",
    values="Weekly Spend",
    sort_columns=True,
).with_columns(
    pl.sum_horizontal(pl.exclude('week')).cast(pl.Float64).alias("Total") # Row total
).sort("week")

weekly_sales_pivot

week,Alpha,Bravo,Charlie,Delta,Other,Total
i16,f64,f64,f64,f64,f64,f64
1,331.459999,247.780003,51.79,17.64,18.75,667.420002
2,567.249997,398.640007,45.43,34.19,23.23,1068.740004
3,497.969999,639.980008,51.11,39.39,14.55,1243.000006
4,1389.960004,472.500004,51.87,85.04,39.779999,2039.150007
5,358.239999,252.540003,40.21,69.54,45.93,766.460002
…,…,…,…,…,…,…
100,692.480006,420.479996,69.600001,54.590001,47.97,1285.120004
101,265.770002,264.659999,64.030001,35.05,26.88,656.390003
102,379.610005,456.78999,43.5,20.4,43.61,943.909995


In [9]:
plotdata = grocery_spend_category.collect()

weekly_plot(dataframe=plotdata, 
            y='Weekly Spend', 
            title='Category - Weekly Revenue', 
            y_axis_label='Spend ($)',
            pct=False,
            legend=False)

JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300, 'stroke': None}, 'axis…

In [10]:
plotdata = grocery_spend_brand.filter(
    pl.col('brand') == 'Alpha'
).collect()

weekly_plot(dataframe=plotdata, 
            y='Weekly Spend', 
            color='brand',
            title='Alpha - Weekly Revenue', 
            y_axis_label='Spend ($)',
            pct=False,
            legend=False)

JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300, 'stroke': None}, 'axis…

In [11]:
plotdata = grocery_spend_brand.filter(
    pl.col('brand') == 'Bravo'
).collect()

weekly_plot(dataframe=plotdata, 
            y='Weekly Spend', 
            color='brand',
            title='Bravo - Weekly Revenue', 
            y_axis_label='Spend ($)',
            pct=False,
            legend=False)

JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300, 'stroke': None}, 'axis…

#### Weekly (Volume) Market Share

In [14]:
# Grocery Volume LazyFrame: Weekly 'volume' - units x weight = volume
grocery_volume = (
    grocery_lf
    .join(
        other=sku_lf,
        left_on="sku_id",
        right_on="sku_id"
    )
    .select(
        ['week', 'units', 'brand', 'weight']
    )
    .with_columns(
        # volume column that is the product of weight of each SKU and the units of SKU sold
        (((pl.col('units') * pl.col('weight'))/1000).cast(pl.Float64)).alias('volume') # # weight from grams to kilograms
    )
    .group_by('week', 'brand')
    .agg(
        pl.col("volume").sum().cast(pl.Float64).alias('Weekly Volume')
    )
    .sort('week')
)

In [17]:
# Weekly (volume) market share
pct_volume =(
    grocery_volume
    .collect()
    .pivot(
        on='brand',
        index='week',
        values='Weekly Volume',
        sort_columns=True
    )
    .with_columns(
        (pl.col("*").exclude("week")) / pl.sum_horizontal(pl.exclude('week')).cast(pl.Float64)
    )
)

pct_volume

week,Alpha,Bravo,Charlie,Delta,Other
i16,f64,f64,f64,f64,f64
1,0.481077,0.355011,0.11194,0.02532,0.026652
2,0.524953,0.371525,0.059391,0.029696,0.014435
3,0.389338,0.508812,0.060676,0.032505,0.008668
4,0.677032,0.230511,0.036686,0.044984,0.010787
5,0.428735,0.324306,0.07574,0.119348,0.051871
…,…,…,…,…,…
100,0.545707,0.31711,0.080381,0.039087,0.017715
101,0.36891,0.40552,0.140805,0.059138,0.025627
102,0.342581,0.543494,0.071147,0.019689,0.023089


In [18]:
pct_volume_plot = (
    grocery_volume
    .group_by('week')
    .agg(
        pl.col('Weekly Volume').sum().alias('Total Weekly Volume')
    )
    .join(
        other=grocery_volume,
        on='week',
    )
    .filter(
        (pl.col('brand') == 'Alpha') |
        (pl.col("brand") == 'Bravo')
    )
    .with_columns(
        # compute brand wise % of total volume sale
        (pl.col('Weekly Volume') / pl.col('Total Weekly Volume')).alias('pct_volume')
    )
).collect()

weekly_plot(dataframe=pct_volume_plot, 
            y='pct_volume', 
            color='brand', 
            title="", 
            y_axis_label="", 
            pct=True, 
            legend=True)

JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300, 'stroke': None}, 'axis…

In [19]:
corr_matrix = (
    pct_volume.select(
        pl.col('*').exclude('week')  # Exclude 'week' column
    )
    .corr()  # Compute the correlation matrix
    .with_columns(
        pl.Series(pct_volume.columns[1:]).alias("index")
    )
)

(
    GT(corr_matrix, rowname_col='index')
    .tab_header(title="Correlation Matrix of Weekly (Volume) Market Share")
    .fmt_number(columns=['Alpha', 'Bravo', 'Charlie', 'Delta', 'Other'])
    .data_color(
        domain=[-1, 1],
        palette=["rebeccapurple", "white", 'orange']
    )
)

GT(_tbl_data=shape: (5, 6)
┌───────────┬───────────┬───────────┬───────────┬───────────┬─────────┐
│ Alpha     ┆ Bravo     ┆ Charlie   ┆ Delta     ┆ Other     ┆ index   │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---     │
│ f64       ┆ f64       ┆ f64       ┆ f64       ┆ f64       ┆ str     │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪═════════╡
│ 1.0       ┆ -0.853061 ┆ -0.34158  ┆ -0.109693 ┆ -0.148193 ┆ Alpha   │
│ -0.853061 ┆ 1.0       ┆ -0.114179 ┆ -0.231641 ┆ -0.104983 ┆ Bravo   │
│ -0.34158  ┆ -0.114179 ┆ 1.0       ┆ 0.131988  ┆ 0.243434  ┆ Charlie │
│ -0.109693 ┆ -0.231641 ┆ 0.131988  ┆ 1.0       ┆ 0.308354  ┆ Delta   │
│ -0.148193 ┆ -0.104983 ┆ 0.243434  ┆ 0.308354  ┆ 1.0       ┆ Other   │
└───────────┴───────────┴───────────┴───────────┴───────────┴─────────┘, _body=<great_tables._gt_data.Body object at 0x000002B40D9A58E0>, _boxhead=Boxhead([ColInfo(var='Alpha', type=<ColInfoTypeEnum.default: 1>, column_label='Alpha', column_align='right', column_width=None), ColInfo(var='Bravo', type=<ColInfoTypeEnum.default: 1>, column_label='Bravo', column_align='right', column_width=None), ColInfo(var='Charlie', type=<ColInfoTypeEnum.default: 1>, column_label='Charlie', column_align='right', column_width=None), ColInfo(var='Delta', type=<ColInfoTypeEnum.default: 1>, column_label='Delta', column_align='right', column_width=None), ColInfo(var='Other', type=<ColInfoTypeEnum.default: 1>, column_label='Other', column_align='right', column_width=None), ColInfo(var='index', type=<ColInfoTypeEnum.stub: 2>, column_label='index', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002B40E258B00>, _spanners=Spanners([]), _heading=Heading(title='Correlation Matrix of Weekly (Volume) Market Share', subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Alpha', rows=[0]), grpname=None, colname='Alpha', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ffa500')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[1]), grpname=None, colname='Alpha', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7c51a8')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[2]), grpname=None, colname='Alpha', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#cbb9dc')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[3]), grpname=None, colname='Alpha', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#eee9f4')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[4]), grpname=None, colname='Alpha', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#e8e1f0')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[0]), grpname=None, colname='Bravo', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7c51a8')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[1]), grpname=None, colname='Bravo', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=Non

#### Annual Sales

In [20]:
# Annual sales by brand
annual_sales = (
    grocery_spend_brand
    .select(
        ['week', 'Weekly Spend', 'brand']
    )
    .with_columns(
        (pl.col("week") / 52).ceil().cast(pl.Int32).alias('year')
    )
    .group_by(['year', 'brand'])
    .agg(
        pl.col("Weekly Spend").sum().cast(pl.Float64).alias('Yearly Sales')
    )
    .sort('year')
)

In [21]:
annual_sales_pivot = annual_sales.collect().pivot(
    on='brand',
    index='year',
    sort_columns=True
).with_columns(
    pl.sum_horizontal(pl.all()).alias("Total") # add totals column
)

(
    GT(annual_sales_pivot, rowname_col="year")
    .tab_header(title="Annual Sales ($) by Brand")
    .tab_stubhead(label="Year")
    .fmt_currency()
    .data_color(
        columns=annual_sales_pivot.columns[:-1],
        domain=[100, 36_000],
        palette=["white", "rebeccapurple"]
    )    
)

GT(_tbl_data=shape: (2, 7)
┌──────┬──────────────┬──────────────┬─────────────┬─────────────┬─────────────┬──────────────┐
│ year ┆ Alpha        ┆ Bravo        ┆ Charlie     ┆ Delta       ┆ Other       ┆ Total        │
│ ---  ┆ ---          ┆ ---          ┆ ---         ┆ ---         ┆ ---         ┆ ---          │
│ i32  ┆ f64          ┆ f64          ┆ f64         ┆ f64         ┆ f64         ┆ f64          │
╞══════╪══════════════╪══════════════╪═════════════╪═════════════╪═════════════╪══════════════╡
│ 1    ┆ 33570.940269 ┆ 28603.350202 ┆ 5120.870036 ┆ 3271.509991 ┆ 1535.229996 ┆ 72102.900494 │
│ 2    ┆ 35250.75023  ┆ 26926.869781 ┆ 3922.68004  ┆ 2820.810008 ┆ 1739.819996 ┆ 70662.930055 │
└──────┴──────────────┴──────────────┴─────────────┴─────────────┴─────────────┴──────────────┘, _body=<great_tables._gt_data.Body object at 0x000002B4053EDAC0>, _boxhead=Boxhead([ColInfo(var='year', type=<ColInfoTypeEnum.stub: 2>, column_label='year', column_align='right', column_width=None), ColInfo(var='Alpha', type=<ColInfoTypeEnum.default: 1>, column_label='Alpha', column_align='right', column_width=None), ColInfo(var='Bravo', type=<ColInfoTypeEnum.default: 1>, column_label='Bravo', column_align='right', column_width=None), ColInfo(var='Charlie', type=<ColInfoTypeEnum.default: 1>, column_label='Charlie', column_align='right', column_width=None), ColInfo(var='Delta', type=<ColInfoTypeEnum.default: 1>, column_label='Delta', column_align='right', column_width=None), ColInfo(var='Other', type=<ColInfoTypeEnum.default: 1>, column_label='Other', column_align='right', column_width=None), ColInfo(var='Total', type=<ColInfoTypeEnum.default: 1>, column_label='Total', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002B40E25DE80>, _spanners=Spanners([]), _heading=Heading(title='Annual Sales ($) by Brand', subtitle=None, preheader=None), _stubhead='Year', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='year', rows=[0]), grpname=None, colname='year', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[1]), grpname=None, colname='year', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[0]), grpname=None, colname='Alpha', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7041a0')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[1]), grpname=None, colname='Alpha', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#69379b')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[0]), grpname=None, colname='Bravo', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#865dae')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[1]), grpname=None, colname='Bravo', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8d67b3')]), StyleInfo(locname=LocBody(columns='Charlie', rows=[0]), grpname=None, colname='Charlie', rownum=0, colnum=None, styles=[CellStyleText(color='#000000'

In [22]:
annual_change = annual_sales_pivot.with_columns(
    pl.col('*').exclude('year').pct_change()
).filter(
    pl.col('year') == 2
).unpivot(
    index="year",  # Keep 'year' as a fixed identifier
    value_name='% Change',  
    variable_name='brand'
)

(
    GT(annual_change, rowname_col='brand')
    .tab_header(title='% Change in Annual Sales')
    .tab_stubhead(label="Brands")
    .fmt_percent()
    .data_color(
        domain=[-0.3, 0.3],
        palette=['orange', 'white', 'rebeccapurple']
    ) 
    .cols_hide('year')   
    .cols_label(brand='Y-o-Y % Change')
)

GT(_tbl_data=shape: (6, 3)
┌──────┬─────────┬───────────┐
│ year ┆ brand   ┆ % Change  │
│ ---  ┆ ---     ┆ ---       │
│ i32  ┆ str     ┆ f64       │
╞══════╪═════════╪═══════════╡
│ 2    ┆ Alpha   ┆ 0.050038  │
│ 2    ┆ Bravo   ┆ -0.058611 │
│ 2    ┆ Charlie ┆ -0.233982 │
│ 2    ┆ Delta   ┆ -0.137765 │
│ 2    ┆ Other   ┆ 0.133263  │
│ 2    ┆ Total   ┆ -0.019971 │
└──────┴─────────┴───────────┘, _body=<great_tables._gt_data.Body object at 0x000002B40E2AA750>, _boxhead=Boxhead([ColInfo(var='year', type=<ColInfoTypeEnum.hidden: 4>, column_label='year', column_align='right', column_width=None), ColInfo(var='brand', type=<ColInfoTypeEnum.stub: 2>, column_label='Y-o-Y % Change', column_align='left', column_width=None), ColInfo(var='% Change', type=<ColInfoTypeEnum.default: 1>, column_label='% Change', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002B40E2A9A30>, _spanners=Spanners([]), _heading=Heading(title='% Change in Annual Sales', subtitle=None, preheader=None), _stubhead='Brands', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='year', rows=[0]), grpname=None, colname='year', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[1]), grpname=None, colname='year', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[2]), grpname=None, colname='year', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[3]), grpname=None, colname='year', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[4]), grpname=None, colname='year', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[5]), grpname=None, colname='year', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[0]), grpname=None, colname='brand', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[1]), grpname=None, colname='brand', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[2]), grpname=None, colname='brand', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[3]), grpname=None, coln

In [23]:
chart = alt.Chart(annual_sales.collect()).mark_bar().encode(
    x=alt.X("brand:N", 
            axis=alt.Axis(labelAngle=0)),
    xOffset="year:N",
    y=alt.Y("Yearly Sales:Q",
            axis=alt.Axis(format="$,.0f")),
    color=alt.Color("year:N"),
).properties(
    width=650,
    height=250,
    title='Year 1 & Year 2 Revenues'
).show()

alt.JupyterChart(chart)

alt.Chart(...)

JupyterChart()

In [24]:
base = alt.Chart(annual_change).encode(
    x=alt.X("brand:N", 
            axis=alt.Axis(labelAngle=0)
    ),
    y=alt.Y("% Change:Q",
            axis=alt.Axis(format=".0%")
    ),
    color=alt.condition(
        alt.datum["% Change"] > 0, # fixed this 
        alt.value("green"),  # The positive color
        alt.value("red")  # The negative color
    ),
    text=alt.Text(
        '% Change',
        format=(".0%")
    )
        
).properties(
    width=650,
    height=250,
    title='Y-o-Y % Change in Revenue'
)

chart = base.mark_bar() + base.mark_text(align='center', 
                                 baseline=alt.expr(alt.expr.if_(alt.datum['% Change'] >= 0, 'bottom', 'top')),
                                 dy=alt.expr(alt.expr.if_(alt.datum['% Change'] >= 0, -2, 2)),
                                 dx=0)

alt.JupyterChart(chart)

JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300}}, 'layer': [{'mark': {…

#### Annual Market Share

In [25]:
market_share = annual_sales_pivot.with_columns(
    pl.col('*').exclude('year') / pl.col('Total')
)

(
    GT(market_share, rowname_col="year")
    .tab_header(title="Annual Market Share (%)")
    .tab_stubhead(label="Year")
    .fmt_percent()
    .cols_hide('Total')
    .data_color(
        domain=[0, 0.5],
        palette=["white", "rebeccapurple"]
    )    
)

GT(_tbl_data=shape: (2, 7)
┌──────┬──────────┬──────────┬──────────┬──────────┬──────────┬───────┐
│ year ┆ Alpha    ┆ Bravo    ┆ Charlie  ┆ Delta    ┆ Other    ┆ Total │
│ ---  ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---      ┆ ---   │
│ i32  ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ f64      ┆ f64   │
╞══════╪══════════╪══════════╪══════════╪══════════╪══════════╪═══════╡
│ 1    ┆ 0.465598 ┆ 0.396702 ┆ 0.071022 ┆ 0.045373 ┆ 0.021292 ┆ 1.0   │
│ 2    ┆ 0.498858 ┆ 0.381061 ┆ 0.055513 ┆ 0.039919 ┆ 0.024621 ┆ 1.0   │
└──────┴──────────┴──────────┴──────────┴──────────┴──────────┴───────┘, _body=<great_tables._gt_data.Body object at 0x000002B40DB96150>, _boxhead=Boxhead([ColInfo(var='year', type=<ColInfoTypeEnum.stub: 2>, column_label='year', column_align='right', column_width=None), ColInfo(var='Alpha', type=<ColInfoTypeEnum.default: 1>, column_label='Alpha', column_align='right', column_width=None), ColInfo(var='Bravo', type=<ColInfoTypeEnum.default: 1>, column_label='Bravo', column_align='right', column_width=None), ColInfo(var='Charlie', type=<ColInfoTypeEnum.default: 1>, column_label='Charlie', column_align='right', column_width=None), ColInfo(var='Delta', type=<ColInfoTypeEnum.default: 1>, column_label='Delta', column_align='right', column_width=None), ColInfo(var='Other', type=<ColInfoTypeEnum.default: 1>, column_label='Other', column_align='right', column_width=None), ColInfo(var='Total', type=<ColInfoTypeEnum.hidden: 4>, column_label='Total', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002B40E2A7FB0>, _spanners=Spanners([]), _heading=Heading(title='Annual Market Share (%)', subtitle=None, preheader=None), _stubhead='Year', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='year', rows=[0]), grpname=None, colname='year', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[1]), grpname=None, colname='year', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[0]), grpname=None, colname='Alpha', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7141a0')]), StyleInfo(locname=LocBody(columns='Alpha', rows=[1]), grpname=None, colname='Alpha', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#663399')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[0]), grpname=None, colname='Bravo', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#865dae')]), StyleInfo(locname=LocBody(columns='Bravo', rows=[1]), grpname=None, colname='Bravo', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8a64b1')]), StyleInfo(locname=LocBody(columns='Charlie', rows=[0]), grpname=None, colname='Charlie', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#e9e2f1')]), StyleInfo(locname=LocBo

In [26]:
annual_change = market_share.drop('Total').with_columns(
    pl.col('*').exclude('year').pct_change()
).filter(
    pl.col('year') == 2
).unpivot(
    index="year",  # Keep 'year' as a fixed identifier
    value_name='% Change',  
    variable_name='brand'
)

(
    GT(annual_change, rowname_col='brand')
    .tab_header(title='% Change in Market Share')
    .tab_stubhead(label="Brands")
    .fmt_percent()
    .data_color(
        domain=[-0.3, 0.3],
        palette=['orange', 'white', 'rebeccapurple']
    ) 
    .cols_hide('year')   
    .cols_label(brand='Y-o-Y % Change')
)

GT(_tbl_data=shape: (5, 3)
┌──────┬─────────┬───────────┐
│ year ┆ brand   ┆ % Change  │
│ ---  ┆ ---     ┆ ---       │
│ i32  ┆ str     ┆ f64       │
╞══════╪═════════╪═══════════╡
│ 2    ┆ Alpha   ┆ 0.071435  │
│ 2    ┆ Bravo   ┆ -0.039428 │
│ 2    ┆ Charlie ┆ -0.218372 │
│ 2    ┆ Delta   ┆ -0.120194 │
│ 2    ┆ Other   ┆ 0.156357  │
└──────┴─────────┴───────────┘, _body=<great_tables._gt_data.Body object at 0x000002B40E0860C0>, _boxhead=Boxhead([ColInfo(var='year', type=<ColInfoTypeEnum.hidden: 4>, column_label='year', column_align='right', column_width=None), ColInfo(var='brand', type=<ColInfoTypeEnum.stub: 2>, column_label='Y-o-Y % Change', column_align='left', column_width=None), ColInfo(var='% Change', type=<ColInfoTypeEnum.default: 1>, column_label='% Change', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002B40E2AEF60>, _spanners=Spanners([]), _heading=Heading(title='% Change in Market Share', subtitle=None, preheader=None), _stubhead='Brands', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='year', rows=[0]), grpname=None, colname='year', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[1]), grpname=None, colname='year', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[2]), grpname=None, colname='year', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[3]), grpname=None, colname='year', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='year', rows=[4]), grpname=None, colname='year', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[0]), grpname=None, colname='brand', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[1]), grpname=None, colname='brand', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[2]), grpname=None, colname='brand', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[3]), grpname=None, colname='brand', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[4]), grpname=None, colname='brand', rownum=4, colnum

In [27]:
base = alt.Chart(annual_change).encode(
    x=alt.X("brand:N", 
            axis=alt.Axis(labelAngle=0)
    ),
    y=alt.Y("% Change:Q",
            axis=alt.Axis(format=".0%")
    ),
    color=alt.condition(
        alt.datum["% Change"] > 0, # fixed this 
        alt.value("green"),  # The positive color
        alt.value("red")  # The negative color
    ),
    text=alt.Text(
        '% Change',
        format=(".0%")
    )
        
).properties(
    width=650,
    height=250,
    title='% Change in Annual Market Share'
)

chart = base.mark_bar() + base.mark_text(align='center', 
                                 baseline=alt.expr(alt.expr.if_(alt.datum['% Change'] >= 0, 'bottom', 'top')),
                                 dy=alt.expr(alt.expr.if_(alt.datum['% Change'] >= 0, -3, 3)),
                                 dx=0)

alt.JupyterChart(chart)

JupyterChart(spec={'config': {'view': {'continuousWidth': 300, 'continuousHeight': 300}}, 'layer': [{'mark': {…

#### Average SKU Price

In [28]:
avg_sku_price = grocery_lf.select(
    ['week', 'sku_id', 'price', 'brand']
).filter(
    (pl.col('week') <= 52) & # pricing in the first year
    (pl.col('brand') == 'Alpha') # Filter by Alpha brand 
).group_by('brand', 'sku_id').agg(
    pl.col('price').cast(pl.Float64).mean()
).drop('brand').sort(
    pl.col('sku_id').cast(pl.Int8)
)

(
    GT(avg_sku_price.collect(), rowname_col='sku_id') 
    .tab_header(title='Alpha - Average SKU Retail Prices')
    .tab_stubhead(label="Brands")
    .fmt_currency()
    .data_color(
        domain=[1, 15],
        palette=['rebeccapurple', 'white', 'orange']
    )  
    .cols_label(price='Average Price ($)')
)

GT(_tbl_data=shape: (18, 2)
┌────────┬──────────┐
│ sku_id ┆ price    │
│ ---    ┆ ---      │
│ cat    ┆ f64      │
╞════════╪══════════╡
│ 4      ┆ 1.84705  │
│ 5      ┆ 3.438107 │
│ 6      ┆ 2.613991 │
│ 7      ┆ 5.099062 │
│ 8      ┆ 12.43    │
│ …      ┆ …        │
│ 18     ┆ 3.274    │
│ 19     ┆ 2.620052 │
│ 20     ┆ 3.43361  │
│ 21     ┆ 2.99     │
│ 22     ┆ 2.09     │
└────────┴──────────┘, _body=<great_tables._gt_data.Body object at 0x000002B40E2A58E0>, _boxhead=Boxhead([ColInfo(var='sku_id', type=<ColInfoTypeEnum.stub: 2>, column_label='sku_id', column_align='center', column_width=None), ColInfo(var='price', type=<ColInfoTypeEnum.default: 1>, column_label='Average Price ($)', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002B40E085DC0>, _spanners=Spanners([]), _heading=Heading(title='Alpha - Average SKU Retail Prices', subtitle=None, preheader=None), _stubhead='Brands', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='sku_id', rows=[0]), grpname=None, colname='sku_id', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[1]), grpname=None, colname='sku_id', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[2]), grpname=None, colname='sku_id', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[3]), grpname=None, colname='sku_id', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[4]), grpname=None, colname='sku_id', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[5]), grpname=None, colname='sku_id', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[6]), grpname=None, colname='sku_id', rownum=6, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[7]), grpname=None, colname='sku_id', rownum=7, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[8]), grpname=None, colname='sku_id', rownum=8, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='sku_id', rows=[9]), grpname=None, colname='sku_id', rownum=9, colnum=None, styles=[CellStyleText(color='#000000

In [29]:
avg_sku_price.drop(pl.col('sku_id')).describe()

statistic,price
str,f64
"""count""",18.0
"""null_count""",0.0
"""mean""",4.042794
"""std""",2.947076
"""min""",1.84705
"""25%""",2.582056
"""50%""",2.99
"""75%""",3.438107
"""max""",12.43


#### Garbage Collect

In [30]:
exceptions = ['grocery_lf', 'sku_lf', 'kiwi_lf', 'In', 'exceptions', 'active_variables']

active_variables = [
    var for var, value in globals().items()
    if not var.startswith('_')   # Exclude variables that start with "_"
    and var not in exceptions    # Exclude variables in the exceptions list
    and isinstance(value, (pl.LazyFrame, pl.DataFrame, pl.Series, alt.Chart, alt.LayerChart, list, int, float, str, np.ndarray, np.int64, np.float32))  # Remove these types only
]

for var in active_variables:
    del globals()[var]
del active_variables, exceptions, var

gc.collect()

767

### Exploring Variation in Buyer Behaviour

#### Panellist-Level Data Preparation

<div style="max-width:600px;margin-left: auto; margin-right: auto;">
<img src="references\Consumer-Panel-Data-Relationship.png" width="600"/>
</div>

In [31]:
def trans_summary(brand, lf, year):
    
    # Primary Step: Filter by Year 1 and Remove Unused Columns
    filtered_lf = lf.filter(
        (pl.col('week') <= (year * 52)) &
        (pl.col('week') >= ((year - 1) * 52))
    ).drop(
        pl.col('week','sku_id')
    )

    # Intermediate Step: Group by trans_id, panel_id, and brand
    group_trans = filtered_lf.drop(
        pl.col('price', 'units')
    ).group_by(
        'trans_id', 'panel_id', 'brand'
    ).n_unique()
    
    if brand == "Category":
        # Panellist-level category transaction summary
        trans = group_trans.group_by(
            'panel_id'
        ).n_unique()
    else:
        # Panellist-level brand transaction summary
        trans = group_trans.filter(
            pl.col('brand') == brand
        ).group_by(
            'panel_id'
        ).n_unique()
    
    return trans.select(
        pl.col('panel_id'),
        pl.col('trans_id').alias('# of Purchases'),
        pl.col('brand').alias('Brands Purchased')
    )

In [32]:
def trans_pivot(lf, year):
    
    # Primary Step: Filter by Year 1 and Remove Unused Columns
    filtered_lf = lf.filter(
        (pl.col('week') <= (year * 52)) &
        (pl.col('week') >= ((year - 1) * 52))
    ).drop(
        pl.col('week','sku_id')
    )

    # Intermediate Step: Group by trans_id, panel_id, and brand
    group_trans = filtered_lf.drop(
        pl.col('price', 'units')
    ).group_by(
        'trans_id', 'panel_id', 'brand'
    ).n_unique()
    
    trans = group_trans.collect().pivot(
        on='brand',
        index='panel_id',
        values='panel_id',
        aggregate_function="len"
        
    ).join(
        other=group_trans.group_by('panel_id').n_unique().drop('brand').collect(),
        on='panel_id'
    ).rename(
        {'trans_id': 'Category'}
    ).drop(
        pl.col('panel_id')
    )
    
    return trans

In [33]:
def spend_summary(brand, lf, year):
    
    group_spend = lf.filter(
        (pl.col('week') <= (year * 52)) &
        (pl.col('week') >= ((year - 1) * 52))
    ).drop(
        pl.col('week','sku_id')
    ).with_columns(
        ((pl.col('units') * pl.col('price')).cast(pl.Float64)).alias('spend')
    )
    
    if brand == "Category":
        # Panellist-level category spend summary
        trans = group_spend.drop(
            pl.col('units', 'price', 'brand')
        ).group_by(
            'panel_id'
        ).agg(
            pl.col('spend').sum()
        ).drop('panel_id')
    else:
        # Panellist-level brand spend summary
        trans = group_spend.drop(
            pl.col('units', 'price')
        ).group_by(
            'panel_id', 'brand'
        ).agg(
            pl.col('spend').sum()
        ).filter(
            pl.col('brand') == brand
        ).drop('brand', 'panel_id')
        
    return trans

#### Examining Purchase Frequency

In [34]:
buyers = trans_pivot(grocery_lf, 1).count().select(
    'Alpha', 'Bravo', 'Charlie', 'Delta', 'Other', 'Category'
).unpivot(
    variable_name='brand',
    value_name='buyers'
)

(
    GT(buyers, rowname_col='brand')
    .tab_header(title='Purchase Occasions by Buyers')
    .tab_stubhead(label="Brands/Category")
    .fmt_integer()
    .data_color(
        rows=['Alpha', 'Bravo', 'Charlie', 'Delta', 'Other'],
        domain=[100, 3_000],
        palette=['white', 'rebeccapurple']
    )  
    .cols_label(buyers='# Of Transactions')
)

GT(_tbl_data=shape: (6, 2)
┌──────────┬────────┐
│ brand    ┆ buyers │
│ ---      ┆ ---    │
│ str      ┆ u32    │
╞══════════╪════════╡
│ Alpha    ┆ 2624   │
│ Bravo    ┆ 2562   │
│ Charlie  ┆ 813    │
│ Delta    ┆ 380    │
│ Other    ┆ 176    │
│ Category ┆ 4574   │
└──────────┴────────┘, _body=<great_tables._gt_data.Body object at 0x000002B40E2AA0C0>, _boxhead=Boxhead([ColInfo(var='brand', type=<ColInfoTypeEnum.stub: 2>, column_label='brand', column_align='left', column_width=None), ColInfo(var='buyers', type=<ColInfoTypeEnum.default: 1>, column_label='# Of Transactions', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002B40E2A6600>, _spanners=Spanners([]), _heading=Heading(title='Purchase Occasions by Buyers', subtitle=None, preheader=None), _stubhead='Brands/Category', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='brand', rows=[0]), grpname=None, colname='brand', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[1]), grpname=None, colname='brand', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[2]), grpname=None, colname='brand', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[3]), grpname=None, colname='brand', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[4]), grpname=None, colname='brand', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='buyers', rows=[0]), grpname=None, colname='buyers', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7a4da6')]), StyleInfo(locname=LocBody(columns='buyers', rows=[1]), grpname=None, colname='buyers', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7d52a8')]), StyleInfo(locname=LocBody(columns='buyers', rows=[2]), grpname=None, colname='buyers', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d9cde6')]), StyleInfo(locname=LocBody(columns='buyers', rows=[3]), grpname=None, colname='buyers', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f0ebf5')]), StyleInfo(locname=LocBody(columns='buyers', rows=[4]), grpname=None, colname='buyers', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None

In [35]:
transactions = trans_pivot(grocery_lf, 1).sum().select(
    'Alpha', 'Bravo', 'Charlie', 'Delta', 'Other', 'Category'
).unpivot(
    variable_name='brand',
    value_name='transactions'
)

(
    GT(transactions, rowname_col='brand')
    .tab_header(title='Purchase Occasions by Buyers')
    .tab_stubhead(label="Brands/Category")
    .fmt_integer()
    .data_color(
        rows=['Alpha', 'Bravo', 'Charlie', 'Delta', 'Other'],
        domain=[400, 10_000],
        palette=[ 'white', 'rebeccapurple']
    )  
    .cols_label(transactions='# Of Transactions')
)

GT(_tbl_data=shape: (6, 2)
┌──────────┬──────────────┐
│ brand    ┆ transactions │
│ ---      ┆ ---          │
│ str      ┆ u32          │
╞══════════╪══════════════╡
│ Alpha    ┆ 9060         │
│ Bravo    ┆ 8255         │
│ Charlie  ┆ 1882         │
│ Delta    ┆ 859          │
│ Other    ┆ 422          │
│ Category ┆ 20030        │
└──────────┴──────────────┘, _body=<great_tables._gt_data.Body object at 0x000002B40DF96030>, _boxhead=Boxhead([ColInfo(var='brand', type=<ColInfoTypeEnum.stub: 2>, column_label='brand', column_align='left', column_width=None), ColInfo(var='transactions', type=<ColInfoTypeEnum.default: 1>, column_label='# Of Transactions', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002B40E2A8F50>, _spanners=Spanners([]), _heading=Heading(title='Purchase Occasions by Buyers', subtitle=None, preheader=None), _stubhead='Brands/Category', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='brand', rows=[0]), grpname=None, colname='brand', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[1]), grpname=None, colname='brand', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[2]), grpname=None, colname='brand', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[3]), grpname=None, colname='brand', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[4]), grpname=None, colname='brand', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='transactions', rows=[0]), grpname=None, colname='transactions', rownum=0, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#7547a3')]), StyleInfo(locname=LocBody(columns='transactions', rows=[1]), grpname=None, colname='transactions', rownum=1, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8258ac')]), StyleInfo(locname=LocBody(columns='transactions', rows=[2]), grpname=None, colname='transactions', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#e7e0ef')]), StyleInfo(locname=LocBody(columns='transactions', rows=[3]), grpname=None, colname='transactions', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f8f5fa')]), StyleInfo(locname=LocBody(columns='transactions', rows=[4]), grpname=None, colname='transactions', rownum=4, colnum=None, styles=[CellStyle

In [36]:
panel_size = 5021

penetration = buyers.with_columns(
    (pl.col('buyers') / panel_size).alias('penetration')
)

ppb = transactions.with_columns(
    (pl.col('transactions') / penetration.get_column('buyers')).alias('ppb')
)

In [37]:
(
    GT(penetration, rowname_col='brand')
    .tab_header(title='Category Penetration by Brand')
    .tab_stubhead(label="Brands/Category")
    .fmt_percent()
    .data_color(
        rows=['Alpha', 'Bravo', 'Charlie', 'Delta', 'Other'],
        domain=[0, 1],
        palette=['white', 'rebeccapurple']
    )  
    .cols_label(penetration='Penetration (%)')
    .cols_hide('buyers')
)

GT(_tbl_data=shape: (6, 3)
┌──────────┬────────┬─────────────┐
│ brand    ┆ buyers ┆ penetration │
│ ---      ┆ ---    ┆ ---         │
│ str      ┆ u32    ┆ f64         │
╞══════════╪════════╪═════════════╡
│ Alpha    ┆ 2624   ┆ 0.522605    │
│ Bravo    ┆ 2562   ┆ 0.510257    │
│ Charlie  ┆ 813    ┆ 0.16192     │
│ Delta    ┆ 380    ┆ 0.075682    │
│ Other    ┆ 176    ┆ 0.035053    │
│ Category ┆ 4574   ┆ 0.910974    │
└──────────┴────────┴─────────────┘, _body=<great_tables._gt_data.Body object at 0x000002B40DA41190>, _boxhead=Boxhead([ColInfo(var='brand', type=<ColInfoTypeEnum.stub: 2>, column_label='brand', column_align='left', column_width=None), ColInfo(var='buyers', type=<ColInfoTypeEnum.hidden: 4>, column_label='buyers', column_align='center', column_width=None), ColInfo(var='penetration', type=<ColInfoTypeEnum.default: 1>, column_label='Penetration (%)', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002B40E25F6B0>, _spanners=Spanners([]), _heading=Heading(title='Category Penetration by Brand', subtitle=None, preheader=None), _stubhead='Brands/Category', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='brand', rows=[0]), grpname=None, colname='brand', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[1]), grpname=None, colname='brand', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[2]), grpname=None, colname='brand', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[3]), grpname=None, colname='brand', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[4]), grpname=None, colname='brand', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='buyers', rows=[0]), grpname=None, colname='buyers', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='buyers', rows=[1]), grpname=None, colname='buyers', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='buyers', rows=[2]), grpname=None, colname='buyers', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='buyers', rows=[3]), grpname=None, colname='buyers', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellSt

In [38]:
(
    GT(ppb, rowname_col='brand')
    .tab_header(title='Penetration per Buyer (PPB)')
    .tab_stubhead(label="Brands/Category")
    .fmt_number()
    .data_color(
        rows=['Alpha', 'Bravo', 'Charlie', 'Delta', 'Other'],
        domain=[0, 5],
        palette=['white', 'rebeccapurple']
    )  
    .cols_label(ppb='PPB')
    .cols_hide('transactions')
)

GT(_tbl_data=shape: (6, 3)
┌──────────┬──────────────┬──────────┐
│ brand    ┆ transactions ┆ ppb      │
│ ---      ┆ ---          ┆ ---      │
│ str      ┆ u32          ┆ f64      │
╞══════════╪══════════════╪══════════╡
│ Alpha    ┆ 9060         ┆ 3.452744 │
│ Bravo    ┆ 8255         ┆ 3.222092 │
│ Charlie  ┆ 1882         ┆ 2.314883 │
│ Delta    ┆ 859          ┆ 2.260526 │
│ Other    ┆ 422          ┆ 2.397727 │
│ Category ┆ 20030        ┆ 4.379099 │
└──────────┴──────────────┴──────────┘, _body=<great_tables._gt_data.Body object at 0x000002B40E2A95B0>, _boxhead=Boxhead([ColInfo(var='brand', type=<ColInfoTypeEnum.stub: 2>, column_label='brand', column_align='left', column_width=None), ColInfo(var='transactions', type=<ColInfoTypeEnum.hidden: 4>, column_label='transactions', column_align='center', column_width=None), ColInfo(var='ppb', type=<ColInfoTypeEnum.default: 1>, column_label='PPB', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002B40E2AEAB0>, _spanners=Spanners([]), _heading=Heading(title='Penetration per Buyer (PPB)', subtitle=None, preheader=None), _stubhead='Brands/Category', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='brand', rows=[0]), grpname=None, colname='brand', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[1]), grpname=None, colname='brand', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[2]), grpname=None, colname='brand', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[3]), grpname=None, colname='brand', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='brand', rows=[4]), grpname=None, colname='brand', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='transactions', rows=[0]), grpname=None, colname='transactions', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='transactions', rows=[1]), grpname=None, colname='transactions', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='transactions', rows=[2]), grpname=None, colname='transactions', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#808080')]), StyleInfo(locname=LocBody(columns='transactions', rows=[3]), grpname=None, colname='transactions', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=N

#### Distribution of Purchase

##### Distribution of Category Purchase Frequency

In [39]:
category_trans_dist = (
    trans_summary('Category', grocery_lf, 1)
    .group_by(pl.col('# of Purchases').cast(pl.Int16))
    .agg(pl.len().cast(pl.Int32).alias("Frequency"))
    .collect() 
    .pipe(
        lambda df: df.vstack(
            pl.DataFrame(
                {
                    "# of Purchases": [0],
                    "Frequency": [panel_size - df["Frequency"].sum()],
                }
            ).with_columns(
                pl.col("# of Purchases").cast(pl.Int16),
                pl.col('Frequency').cast(pl.Int32)
                )
        )
    )
    .sort(by='# of Purchases')
    .with_columns(
        (pl.col('Frequency') / pl.col('Frequency').sum()).cast(pl.Float64).alias("% of Total")
    )    
)

(
    GT(category_trans_dist, rowname_col='# of Purchases')
    .tab_header(title='Category Purchase Distribution')
    .tab_stubhead(label="# of Purchases")
    .fmt_integer(columns='Frequency')
    .fmt_percent(columns='% of Total')
    .data_color(
        columns=['% of Total'],
        domain=[0, 1],
        palette=['white', 'rebeccapurple']
    )  
)

GT(_tbl_data=shape: (24, 3)
┌────────────────┬───────────┬────────────┐
│ # of Purchases ┆ Frequency ┆ % of Total │
│ ---            ┆ ---       ┆ ---        │
│ i16            ┆ i32       ┆ f64        │
╞════════════════╪═══════════╪════════════╡
│ 0              ┆ 447       ┆ 0.089026   │
│ 1              ┆ 667       ┆ 0.132842   │
│ 2              ┆ 759       ┆ 0.151165   │
│ 3              ┆ 757       ┆ 0.150767   │
│ 4              ┆ 612       ┆ 0.121888   │
│ …              ┆ …         ┆ …          │
│ 19             ┆ 2         ┆ 0.000398   │
│ 20             ┆ 4         ┆ 0.000797   │
│ 22             ┆ 1         ┆ 0.000199   │
│ 25             ┆ 1         ┆ 0.000199   │
│ 27             ┆ 1         ┆ 0.000199   │
└────────────────┴───────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x000002B40E0E3B60>, _boxhead=Boxhead([ColInfo(var='# of Purchases', type=<ColInfoTypeEnum.stub: 2>, column_label='# of Purchases', column_align='right', column_width=None), ColInfo(var='Frequency', type=<ColInfoTypeEnum.default: 1>, column_label='Frequency', column_align='right', column_width=None), ColInfo(var='% of Total', type=<ColInfoTypeEnum.default: 1>, column_label='% of Total', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002B40E108D70>, _spanners=Spanners([]), _heading=Heading(title='Category Purchase Distribution', subtitle=None, preheader=None), _stubhead='# of Purchases', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='% of Total', rows=[0]), grpname=None, colname='% of Total', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f1edf6')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[1]), grpname=None, colname='% of Total', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ebe4f1')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[2]), grpname=None, colname='% of Total', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#e8e0f0')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[3]), grpname=None, colname='% of Total', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#e8e0f0')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[4]), grpname=None, colname='% of Total', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ece6f3')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[5]), grpname=None, colname='% of Total', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#efeaf5')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[6]), grpname=None, colname='% of Total', rownum=6, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f3f0f7')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[7]), grpname=None, colname='% of Total', rownum=7, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, sty

In [40]:
labels = [str(i) if i < 15 else '15+' for i in range(16)]

category_trans_dist = category_trans_dist.with_columns(
    pl.col('# of Purchases').cut(np.arange(0,15,1), labels=labels).alias('cut')    
)
chart = alt.Chart(category_trans_dist).mark_bar().encode(
    x=alt.X("cut:O", 
            axis=alt.Axis(labelAngle=0, title="Number of Category Transactions"), sort=labels),
    y=alt.Y("% of Total:Q",
            axis=alt.Axis(format=".0%", title="% of Households")), 
).properties(
    width=650,
    height=250,
    title={
        'text': 'Category Transaction Distribution (%)',
        'subtitle': 'Distribution of category purchasing in year 1'
    }
).show()

alt.JupyterChart(chart)

alt.Chart(...)

JupyterChart()

##### Distribution of Brand Purchase Frequency

In [41]:
alpha_trans_dist = (
    trans_summary('Alpha', grocery_lf, 1)
    .group_by(pl.col('# of Purchases').cast(pl.Int16))
    .agg(pl.len().cast(pl.Int32).alias("Frequency"))
    .collect() 
    .pipe(
        lambda df: df.vstack(
            pl.DataFrame(
                {
                    "# of Purchases": [0],
                    "Frequency": [panel_size - df["Frequency"].sum()],
                }
            ).with_columns(
                pl.col("# of Purchases").cast(pl.Int16),
                pl.col('Frequency').cast(pl.Int32)
                )
        )
    )
    .sort(by='# of Purchases')
    .with_columns(
        (pl.col('Frequency') / pl.col('Frequency').sum()).cast(pl.Float64).alias("% of Total")
    )    
)

(
    GT(alpha_trans_dist, rowname_col='# of Purchases')
    .tab_header(title='Alpha Purchase Distribution')
    .tab_stubhead(label="# of Purchases")
    .fmt_integer(columns='Frequency')
    .fmt_percent(columns='% of Total')
    .data_color(
        columns=['% of Total'],
        domain=[0, 1],
        palette=['white', 'rebeccapurple']
    )  
)

GT(_tbl_data=shape: (19, 3)
┌────────────────┬───────────┬────────────┐
│ # of Purchases ┆ Frequency ┆ % of Total │
│ ---            ┆ ---       ┆ ---        │
│ i16            ┆ i32       ┆ f64        │
╞════════════════╪═══════════╪════════════╡
│ 0              ┆ 2397      ┆ 0.477395   │
│ 1              ┆ 733       ┆ 0.145987   │
│ 2              ┆ 517       ┆ 0.102968   │
│ 3              ┆ 400       ┆ 0.079665   │
│ 4              ┆ 277       ┆ 0.055168   │
│ …              ┆ …         ┆ …          │
│ 14             ┆ 12        ┆ 0.00239    │
│ 15             ┆ 6         ┆ 0.001195   │
│ 16             ┆ 3         ┆ 0.000597   │
│ 18             ┆ 1         ┆ 0.000199   │
│ 20             ┆ 1         ┆ 0.000199   │
└────────────────┴───────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x000002B40E2B7620>, _boxhead=Boxhead([ColInfo(var='# of Purchases', type=<ColInfoTypeEnum.stub: 2>, column_label='# of Purchases', column_align='right', column_width=None), ColInfo(var='Frequency', type=<ColInfoTypeEnum.default: 1>, column_label='Frequency', column_align='right', column_width=None), ColInfo(var='% of Total', type=<ColInfoTypeEnum.default: 1>, column_label='% of Total', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002B40E085E50>, _spanners=Spanners([]), _heading=Heading(title='Alpha Purchase Distribution', subtitle=None, preheader=None), _stubhead='# of Purchases', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='% of Total', rows=[0]), grpname=None, colname='% of Total', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#b69ece')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[1]), grpname=None, colname='% of Total', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#e9e1f0')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[2]), grpname=None, colname='% of Total', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#efeaf4')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[3]), grpname=None, colname='% of Total', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f3eff7')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[4]), grpname=None, colname='% of Total', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f7f4f9')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[5]), grpname=None, colname='% of Total', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#f8f6fa')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[6]), grpname=None, colname='% of Total', rownum=6, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#fbf9fc')]), StyleInfo(locname=LocBody(columns='% of Total', rows=[7]), grpname=None, colname='% of Total', rownum=7, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=

In [42]:
labels = [str(i) if i < 10 else '10+' for i in range(11)]

alpha_trans_dist = alpha_trans_dist.with_columns(
    pl.col('# of Purchases').cut(np.arange(0,10,1), labels=labels).alias('cut')    
)
chart = alt.Chart(alpha_trans_dist).mark_bar().encode(
    x=alt.X("cut:O", 
            axis=alt.Axis(labelAngle=0, title="Number of Transactions"), sort=labels),
    y=alt.Y("% of Total:Q",
            axis=alt.Axis(format=".0%", title="% of Households")), 
).properties(
    width=650,
    height=250,
    title={
        'text': 'Alpha Transaction Distribution (%)',
        'subtitle': 'Distribution of purchase frequency for Alpha in year 1'
    }
).show()

alt.JupyterChart(chart)

alt.Chart(...)

JupyterChart()

#### Distribution of Spend

##### Distribution of Category Spend

In [43]:
category_spend = spend_summary('Category', grocery_lf, 1)

category_spend.describe()

statistic,spend
str,f64
"""count""",4574.0
"""null_count""",0.0
"""mean""",15.763424
"""std""",12.876024
"""min""",1.27
"""25%""",6.76
"""50%""",12.57
"""75%""",20.74
"""max""",166.699999


In [49]:
percentiles = np.arange(5, 96, 5)

percentile_values = np.percentile(category_spend.select('spend').collect(), percentiles)

percentile_df = pl.DataFrame({'Percentile Level': percentiles,
                              'Percentile': percentile_values})

(
    GT(percentile_df, rowname_col='Percentile Level')
    .tab_header(title='Alpha Spend Percentile')
    .tab_stubhead(label="Percentile Level")
    .fmt_number()
    .data_color(
        domain=[0, 40],
        palette=['white', 'rebeccapurple']
    )  
)

GT(_tbl_data=shape: (19, 2)
┌──────────────────┬────────────┐
│ Percentile Level ┆ Percentile │
│ ---              ┆ ---        │
│ i32              ┆ f64        │
╞══════════════════╪════════════╡
│ 5                ┆ 2.69       │
│ 10               ┆ 3.39       │
│ 15               ┆ 4.67       │
│ 20               ┆ 5.88       │
│ 25               ┆ 6.76       │
│ …                ┆ …          │
│ 75               ┆ 20.7375    │
│ 80               ┆ 23.43      │
│ 85               ┆ 26.8805    │
│ 90               ┆ 31.810001  │
│ 95               ┆ 39.7185    │
└──────────────────┴────────────┘, _body=<great_tables._gt_data.Body object at 0x000002B40E35E420>, _boxhead=Boxhead([ColInfo(var='Percentile Level', type=<ColInfoTypeEnum.stub: 2>, column_label='Percentile Level', column_align='right', column_width=None), ColInfo(var='Percentile', type=<ColInfoTypeEnum.default: 1>, column_label='Percentile', column_align='right', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x000002B40E35FCB0>, _spanners=Spanners([]), _heading=Heading(title='Alpha Spend Percentile', subtitle=None, preheader=None), _stubhead='Percentile Level', _source_notes=[], _footnotes=[], _styles=[StyleInfo(locname=LocBody(columns='Percentile Level', rows=[0]), grpname=None, colname='Percentile Level', rownum=0, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#ece6f2')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[1]), grpname=None, colname='Percentile Level', rownum=1, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#d9cce6')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[2]), grpname=None, colname='Percentile Level', rownum=2, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#c6b2d9')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[3]), grpname=None, colname='Percentile Level', rownum=3, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#b299cc')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[4]), grpname=None, colname='Percentile Level', rownum=4, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#9f80bf')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[5]), grpname=None, colname='Percentile Level', rownum=5, colnum=None, styles=[CellStyleText(color='#000000', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#8c66b2')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[6]), grpname=None, colname='Percentile Level', rownum=6, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#794ca6')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[7]), grpname=None, colname='Percentile Level', rownum=7, colnum=None, styles=[CellStyleText(color='#FFFFFF', font=None, size=None, align=None, v_align=None, style=None, weight=None, stretch=None, decorate=None, transform=None, whitespace=None), CellStyleFill(color='#663399')]), StyleInfo(locname=LocBody(columns='Percentile Level', rows=[8]), grpname=None, colnam